# 1. Configura Kubernetes Authentication

In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

Importamos la dirección, localización del fichero de la CA de Vault así como el Vault token usando [python-dotenv](https://pypi.org/project/python-dotenv/). De esta forma evitamos filtrar el root token de Vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')


Habilitamos Kubernetes Auth Method usando el path por defecto (```kubernetes```)

In [ ]:
! vault auth enable kubernetes

Configuramos Vault para que hable con la API del clúster local usando su propia CA. La SA que usa Vault tiene permisos para verificar los tokens presnetados contra la API de Vault contra la API de Kubernetes.

In [ ]:
%%bash
HOST=$(kubectl get svc kubernetes -o json | jq -r .spec.clusterIP)
PORT=$(kubectl get svc kubernetes -o json | jq -r '.spec.ports[0].port')
vault write auth/kubernetes/config kubernetes_host=https://$HOST:$PORT

Creamos un role que será usado por las aplicaciones vía VSO para acceder a los secretos de Vault

In [ ]:
%%bash

vault write auth/kubernetes/role/role \
    bound_service_account_names=default,internal-app \
    bound_service_account_namespaces=static,database,pki,agent,test,default \
    audience="https://kubernetes.default.svc.cluster.local" \
    policies=devk8s \
    ttl=30m

Y la política asociada al role que hemos creado en el paso previos

In [ ]:
%%bash
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
EOF

### Creamos un secreto estático en Vault

Habilitamos la engine

In [ ]:
! vault secrets enable -path=kvv2 kv-v2

Creamos un secreto

In [ ]:
! vault kv put kvv2/webapp/config username="static-user" password="static-password"

# 2. Instalando el VSO usando helm. [Referencia](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/installation)

In [ ]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update
# Los recursos se instalaran en el namespace vault-secrets-operator
helm install --version 1.0.1 --create-namespace --namespace vault-secrets-operator vault-secrets-operator hashicorp/vault-secrets-operator

In [ ]:
! kubectl get events -n vault-secrets-operator

In [ ]:
! kubectl get pods -n vault-secrets-operator

### Instalamos los CRDs

[VaultConnection](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultconnectionspec)

In [ ]:
%%bash

cat > ${WORKDIR}/vso_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultConnection
metadata:
  namespace: vault-secrets-operator
  name: example
spec:
  address: https://vault.vault.svc.cluster.local:8200
  skipTLSVerify: true

EOF

kubectl apply -f ${WORKDIR}/vso_crd.yaml

Verify VaultConnection is being deployed

In [ ]:
! kubectl describe VaultConnection example -n vault-secrets-operator

[VaultAuth](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultauthspec)

In [ ]:
%%bash
cat > ${WORKDIR}/vaultauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault-secrets-operator
  name: example
spec:
  vaultConnectionRef: example
  allowedNamespaces: ["*"]
  method: kubernetes
  mount: kubernetes

  kubernetes:
    # role to use when authenticating to Vault
    role: role
    serviceAccount: default

EOF
kubectl apply -f ${WORKDIR}/vaultauth_crd.yaml


Verify VaultAuth is being deployed

In [ ]:
! kubectl describe VaultAuth example -n vault-secrets-operator

[VaultStaticSecret](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultstaticsecretspec)


In [ ]:
! kubectl create ns static

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: static
  name: example
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: kvv2
  type: kv-v2
  path: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF

kubectl apply -f ${WORKDIR}/static_secret.yaml

Verificamos que el secreto se sincroniza

In [ ]:
! kubectl describe VaultStaticSecret example -n static

Por último veriquemos que podemos leer el secreto (decodificar el base64)

In [ ]:
%%bash
echo "USERNAME: $(kubectl get secret static-secret1 -n static -o json | jq -r .data.username | base64 -d)"
echo "PASSWORD: $(kubectl get secret static-secret1 -n static -o json | jq -r .data.password | base64 -d)"

In [ ]:
%%bash
cat > ${WORKDIR}/mypod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod
  namespace: static
spec:
  containers:
  - name: mypod
    image: nginx
    env:
    - name: USERNAME
      valueFrom:
        secretKeyRef:
          name: static-secret1
          key: username
    - name: PASSWORD
      valueFrom:
        secretKeyRef:
          name: static-secret1
          key: password
EOF

kubectl apply -f ${WORKDIR}/mypod.yaml
sleep 10



In [ ]:
! kubectl exec mypod -n static -- env | grep -E 'USERNAME|PASSWORD'

## Rollout Restart

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/static_secret2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: static
  name: example2
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: kvv2
  type: kv-v2
  hmacSecretData: true          # Enable HMAC signing of secret data. Verify changes to secret data.
  rolloutRestartTargets:        # Restart target deployments when secret data changes.
    - kind: Deployment
      name: mydeployment
  path: webapp/config
  refreshAfter: 60s             # Check for updates every 60 seconds. Not using events
  destination:
    create: true
    name: static-secret2
EOF

kubectl apply -f ${WORKDIR}/static_secret2.yaml

In [ ]:
! kubectl describe VaultStaticSecret example2 -n static

In [ ]:
%%bash
cat > ${WORKDIR}/mydeployment.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mydeployment
  namespace: static
spec:
  replicas: 2
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
      - name: mypod
        image: nginx
        env:
        - name: USERNAME
          valueFrom:
            secretKeyRef:
              name: static-secret2
              key: username
        - name: PASSWORD
          valueFrom:
            secretKeyRef:
              name: static-secret2
              key: password
EOF

kubectl apply -f ${WORKDIR}/mydeployment.yaml

In [ ]:
! kubectl get deployments -n static

In [ ]:
! kubectl get deployment mydeployment -n static -o yaml

In [ ]:
! kubectl exec deploy/mydeployment -n static -- env | grep -E 'USERNAME|PASSWORD'

In [ ]:
! vault kv put kvv2/webapp/config username="static-user2" password="static-password2"

In [ ]:
! kubectl describe deployment mydeployment -n static

In [ ]:
! kubectl get rs -n static -l app=myapp

In [ ]:
! kubectl get pods -n static -l app=myapp

## Instant updates for VaultStaticSecrets

### Set Event permissions
https://developer.hashicorp.com/vault/docs/concepts/events#policies

In [ ]:
%%bash

vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read", "list", "subscribe"]
  subscribe_event_types = ["*"]                     # https://developer.hashicorp.com/vault/docs/concepts/events#event-types
}
path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
EOF

### Create VaultStaticSecret
https://developer.hashicorp.com/vault/docs/deploy/kubernetes/vso/sources/vault/instant-updates#step-2-enable-instant-updates-on-the-vaultstaticsecret

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/static_secret_event.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: static
  name: example-event
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: kvv2
  type: kv-v2
  path: webapp/config
  refreshAfter: 1h
  destination:
    create: true
    name: static-secret-event
  syncConfig:
    instantUpdates: true
EOF

kubectl apply -f ${WORKDIR}/static_secret_event.yaml

In [ ]:
! kubectl describe VaultStaticSecret example-event -n static

In [ ]:
! vault kv put kvv2/webapp/config username="static-user12" password="static-password12"

In [ ]:
! kubectl describe VaultStaticSecret example-event -n static

In [ ]:
%%bash
kubectl get secret static-secret-event -n static -o json | jq -r .data.username | base64 -d
echo ""
kubectl get secret static-secret-event -n static -o json | jq -r .data.password | base64 -d

### Dynamic Secrets

Actualizar la políticas con permisos de lectura al path de la database

In [ ]:
%%bash

vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read", "list", "subscribe"]
  subscribe_event_types = ["*"]                     # https://developer.hashicorp.com/vault/docs/concepts/events#event-types
}
path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
EOF

[VaultDynamicSecret](https://developer.hashicorp.com/vault/docs/platform/k8s/vso/api-reference#vaultdynamicsecretspec)

In [ ]:
! kubectl create ns database

In [ ]:
%%bash

cat > ${WORKDIR}/dynamic_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultDynamicSecret
metadata:
  namespace: database
  name: db-secret
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: database
  path: creds/readonly
  destination:
    create: true
    name: db-secret
EOF

kubectl apply -f ${WORKDIR}/dynamic_secret.yaml


Verificar que el secreto ha sido sincronizado

In [ ]:
%%bash
sleep 5
kubectl get secret db-secret -n database -o yaml

Ahora decodificando el base64

In [ ]:
%%bash
echo "USERNAME: $(kubectl get secret db-secret -n database -o json | jq -r .data.username | base64 -d)"
echo "PASSWORD: $(kubectl get secret db-secret -n database -o json | jq -r .data.password | base64 -d)"

### Montemos el secreto en un POD

In [ ]:
%%bash
cat > ${WORKDIR}/mypod_db.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypoddb
  namespace: database
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: db-secret
      optional: true
EOF


# Despliega el POD
kubectl apply -f ${WORKDIR}/mypod_db.yaml

Verifiquemos que el secreto se monta en el pod

In [ ]:
%%bash
# Espera a que despliegue
sleep 10 
# Chequea secretos
kubectl exec mypoddb -n database  -- ls /etc/foo/
echo ""
echo "------"
kubectl exec mypoddb -n database -- cat /etc/foo/_raw

Si esperamos 5 minutos (por la configuración del role) veremos un nuevo secreto en el volumen del POD

In [ ]:
%%bash
sleep 300 
kubectl exec mypoddb -n database -- cat /etc/foo/_raw

### RolloutRestart

In [ ]:
%%bash

cat > ${WORKDIR}/dynamic_secret2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultDynamicSecret
metadata:
  namespace: database
  name: db-secret-restart
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: database
  path: creds/readonly
  renewalPercent: 90            # Renew secret when 90% of its lease time has elapsed.
  revoke: true
  rolloutRestartTargets:        # Restart target deployments when secret data changes.
    - kind: Deployment
      name: mydbreader
  destination:
    create: true
    name: db-secret-restart
EOF

kubectl apply -f ${WORKDIR}/dynamic_secret2.yaml

In [ ]:
! kubectl describe VaultDynamicSecret db-secret-restart -n database

In [ ]:
%%bash
kubectl get secret db-secret -n database -o json | jq -r .data.username | base64 -d
echo ""
kubectl get secret db-secret -n database -o json | jq -r .data.password | base64 -d
echo ""
echo "sleep 300 seconds to wait for renewal"
echo ""
sleep 300
kubectl get secret db-secret -n database -o json | jq -r .data.username | base64 -d
echo ""
kubectl get secret db-secret -n database -o json | jq -r .data.password | base64 -d

### Create Deployment to test restart on secret change

In [ ]:
%%bash
cat > ${WORKDIR}/mydbreader.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mydbreader
  namespace: database
spec:
  replicas: 2
  selector:
    matchLabels:
      app: mydbreader
  template:
    metadata:
      labels:
        app: mydbreader
    spec:
      containers:
      - name: nginx
        image: nginx:latest
        env:
        - name: USERNAME
          valueFrom:
            secretKeyRef:
              name: db-secret-restart
              key: username
        - name: PASSWORD
          valueFrom:
            secretKeyRef:
              name: db-secret-restart
              key: password
        command: ["/bin/sh"]
        args:
          - -c
          - |
            cat > /usr/share/nginx/html/index.html <<HTML
            <!DOCTYPE html>
            <html>
            <head>
              <title>Database Credentials</title>
              <style>
                body { font-family: Arial, sans-serif; margin: 40px; background-color: #f0f0f0; }
                .container { background-color: white; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
                h1 { color: #333; }
                .credential { margin: 10px 0; padding: 10px; background-color: #e8f4f8; border-left: 4px solid #2196F3; }
                .label { font-weight: bold; color: #555; }
                .value { color: #2196F3; font-family: monospace; }
              </style>
            </head>
            <body>
              <div class="container">
                <h1>Database Credentials from Vault</h1>
                <div class="credential">
                  <span class="label">Username:</span>
                  <span class="value">\$USERNAME</span>
                </div>
                <div class="credential">
                  <span class="label">Password:</span>
                  <span class="value">\$PASSWORD</span>
                </div>
                <p><small>Pod: \$(hostname)</small></p>
              </div>
            </body>
            </html>
            HTML
            nginx -g 'daemon off;'
        ports:
        - containerPort: 80
          name: http
---
apiVersion: v1
kind: Service
metadata:
  name: mydbreader
  namespace: database
spec:
  selector:
    app: mydbreader
  ports:
  - port: 80
    targetPort: 80
    protocol: TCP
  type: ClusterIP
EOF

kubectl apply -f ${WORKDIR}/mydbreader.yaml

In [ ]:
! kubectl get deployments -n database mydbreader -o yaml 

In [ ]:
%%bash
echo "Waiting for deployment to be ready..."
kubectl wait --for=condition=available --timeout=300s deployment/mydbreader -n database
echo ""
echo "Deployment status:"
kubectl get deployment mydbreader -n database
echo ""
echo "Pods:"
kubectl get pods -n database -l app=mydbreader

In [ ]:
%%bash
echo "Service details:"
kubectl get svc mydbreader -n test
echo ""
echo "Testing the service (fetching the HTML page):"
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -n test -- curl -s http://mydbreader.test.svc.cluster.local

Verificar que el deployment se reinicia automáticamente cuando cambian las credenciales dinámicas

In [ ]:
%%bash
echo "Current credentials displayed in the app:"
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -n database -- curl -s http://mydbreader.database.svc.cluster.local | grep -A 2 "Username:\|Password:"
echo ""
echo "Current pod names (note the age):"
kubectl get pods -n test -l app=mydbreader
echo ""
echo "Waiting for credential renewal (this may take almost 5 minutes based on renewalPercent: 90)..."

In [ ]:
%%bash
sleep 600
echo "Current credentials displayed in the app:"
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -n database -- curl -s http://mydbreader.database.svc.cluster.local | grep -A 2 "Username:\|Password:"
echo ""
echo "Current pod names (note the age):"
kubectl get pods -n database -l app=mydbreader
echo ""
echo "Waiting for credential renewal (this may take almost 9 minutes based on renewalPercent: 90)..."

In [ ]:
! kubectl describe deploy -n database mydbreader

In [ ]:
! kubectl describe VaultDynamicSecret db-secret-restart -n database

## VaultPKISecret
https://developer.hashicorp.com/vault/docs/deploy/kubernetes/vso/api-reference#vaultpkisecret

### Modify permissions

In [ ]:
%%bash

vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read", "list", "subscribe"]
  subscribe_event_types = ["*"]                     # https://developer.hashicorp.com/vault/docs/concepts/events#event-types
}
path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
path "pki_int/issue/example-dot-com" {
  capabilities = [ "update" ]
}
path "pki_int/revoke" {
  capabilities = [ "update" ]
}

EOF

In [ ]:
! kubectl create ns pki

In [ ]:
%%bash
cat > ${WORKDIR}/pki_secret.yaml <<EOF
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultPKISecret
metadata:
  namespace: pki
  name: pki1
spec:
  vaultAuthRef: vault-secrets-operator/example
  mount: pki_int
  role: example-dot-com
  commonName: opus111.example.com
  altNames:
    - www.opus111.example.com
  revoke: true
  rolloutRestartTargets:        # Restart target deployments when secret data changes.
    - kind: Deployment
      name: mypkiapp
  format: pem
  expiryOffset: 30s             # Time before expiry to renew the certificate
  ttl: 300s                     # Duration of the certificate      
  destination:
    create: true
    name: pki1
EOF

kubectl apply -f ${WORKDIR}/pki_secret.yaml

In [ ]:
! kubectl describe VaultPKISecret pki1 -n pki

In [ ]:
%%bash
kubectl get secret pki1 -n pki -o yaml

# Mount secret into Deployment

In [ ]:
%%bash
cat > ${WORKDIR}/mypkiapp.yaml <<'EOF'
apiVersion: v1
kind: ConfigMap
metadata:
  name: nginx-config
  namespace: pki
data:
  nginx.conf: |
    events {
        worker_connections 1024;
    }
    http {
        server {
            listen 443 ssl;
            server_name opus111.example.com;
            
            ssl_certificate /etc/nginx/certs/tls.crt;
            ssl_certificate_key /etc/nginx/certs/tls.key;
            
            ssl_protocols TLSv1.2 TLSv1.3;
            ssl_ciphers HIGH:!aNULL:!MD5;
            
            location / {
                root /usr/share/nginx/html;
                index index.html;
            }
        }
    }
  index.html: |
    <!DOCTYPE html>
    <html>
    <head>
        <title>PKI Certificate Info</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                min-height: 100vh;
            }
            .container {
                max-width: 900px;
                margin: 0 auto;
                background: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 10px 40px rgba(0,0,0,0.3);
            }
            h1 {
                color: #667eea;
                text-align: center;
                margin-bottom: 30px;
            }
            .cert-section {
                margin: 20px 0;
                padding: 20px;
                background: #f8f9fa;
                border-left: 4px solid #667eea;
                border-radius: 5px;
            }
            .cert-label {
                font-weight: bold;
                color: #555;
                margin-bottom: 5px;
            }
            .cert-value {
                color: #333;
                font-family: monospace;
                background: white;
                padding: 10px;
                border-radius: 3px;
                word-break: break-all;
            }
            .success {
                background: #d4edda;
                border-color: #28a745;
                color: #155724;
                padding: 15px;
                border-radius: 5px;
                margin: 20px 0;
            }
            .command {
                background: #2d3748;
                color: #68d391;
                padding: 15px;
                border-radius: 5px;
                margin: 20px 0;
                font-family: monospace;
                overflow-x: auto;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Vault PKI Certificate Information</h1>
            <div class="success">
                This connection is secured with a certificate issued by HashiCorp Vault PKI
            </div>
            <div class="cert-section">
                <div class="cert-label">Common Name</div>
                <div class="cert-value">opus111.example.com</div>
            </div>
            <div class="cert-section">
                <div class="cert-label">Subject Alternative Names</div>
                <div class="cert-value">opus111.example.com, www.opus111.example.com</div>
            </div>
            <div class="cert-section">
                <div class="cert-label">Certificate TTL</div>
                <div class="cert-value">60 seconds (auto-rotates via VSO)</div>
            </div>
            <div class="cert-section">
                <div class="cert-label">Vault PKI Role</div>
                <div class="cert-value">example-dot-com</div>
            </div>
            <div class="cert-section">
                <div class="cert-label">Vault Mount Path</div>
                <div class="cert-value">pki_int</div>
            </div>
        </div>
    </body>
    </html>
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mypkiapp
  namespace: pki
spec:
  replicas: 5
  selector:
    matchLabels:
      app: mypkiapp
  template:
    metadata:
      labels:
        app: mypkiapp
    spec:
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 443
          name: https
        volumeMounts:
        - name: certs
          mountPath: /etc/nginx/certs
          readOnly: true
        - name: nginx-config
          mountPath: /etc/nginx/nginx.conf
          subPath: nginx.conf
          readOnly: true
        - name: html
          mountPath: /usr/share/nginx/html/index.html
          subPath: index.html
          readOnly: true
      volumes:
      - name: certs
        secret:
          secretName: pki1
          items:
          - key: certificate
            path: tls.crt
          - key: private_key
            path: tls.key
      - name: nginx-config
        configMap:
          name: nginx-config
      - name: html
        configMap:
          name: nginx-config
---
apiVersion: v1
kind: Service
metadata:
  name: mypkiapp
  namespace: pki
spec:
  selector:
    app: mypkiapp
  ports:
  - port: 443
    targetPort: 443
    protocol: TCP
    name: https
  type: ClusterIP
EOF

kubectl apply -f ${WORKDIR}/mypkiapp.yaml

In [ ]:
%%bash
echo "Waiting for deployment to be ready..."
kubectl wait --for=condition=available --timeout=60s deployment/mypkiapp -n pki
echo ""
echo "Deployment status:"
kubectl get deployment mypkiapp -n pki
echo ""
echo "Pods:"
kubectl get pods -n pki -l app=mypkiapp
echo ""
echo "Service:"
kubectl get svc mypkiapp -n pki

In [ ]:
%%bash
# check deployment annotations
kubectl get deploy -n pki mypkiapp -o yaml 

Verificar el certificado montado en el deployment

In [ ]:
%%bash
echo "Certificate details from the mounted volume:"
kubectl exec -n pki deploy/mypkiapp -- openssl x509 -in /etc/nginx/certs/tls.crt -text -noout | grep -A 5 "Subject:\|Issuer:\|Validity\|DNS:"

Probar la conexión TLS al servicio

In [ ]:
%%bash

echo "Testing HTTPS connection to the service:"
kubectl run -it --rm curl-test --image=curlimages/curl --restart=Never -n pki -- curl -k https://mypkiapp.pki.svc.cluster.local | head -50

Verificar que el certificado se renueva automáticamente (espera 4 mins y medio por el TTL)

In [ ]:
%%bash
echo "Current certificate serial number:"
kubectl exec -n pki deploy/mypkiapp -- openssl x509 -in /etc/nginx/certs/tls.crt -serial -noout
echo ""
echo "Current pods age:"
kubectl get pods -n pki -l app=mypkiapp
echo ""
echo "Waiting 270 seconds for certificate renewal and rollout restart..."
sleep 270
echo ""
echo "New certificate serial number (should be different):"
kubectl exec -n pki deploy/mypkiapp -- openssl x509 -in /etc/nginx/certs/tls.crt -serial -noout
echo ""
echo "New pods (check age - should be recently restarted):"
kubectl get pods -n pki -l app=mypkiapp

In [ ]:
! kubectl describe deploy -n pki mypkiapp

## Create forwarding port to access deployment

In [ ]:
%%bash

# proxy connection towards nginx

kubectl -n pki port-forward --address 0.0.0.0 service/mypkiapp 8843:443